# NMCT-Box Jupyter demo

- Via de Jupyter webinterface kan je live code uitvoeren op de NMCT-box. 
- Het resultaat kan je zo opslaan als "notebook", maar ook als zuiver Python-bestand
- Of exporteren als HTML, Markdown, PDF,... 

## NMCT Box basics

In [1]:
import nmct.box

acc = nmct.box.get_accelerometer()
print(acc.measure())
print(acc.tilt())


Acceleration(x=10.5912, y=0.5492, z=-1.1768)
Tilt(roll=2.9502546890263526, pitch=83.00954422765167)


In [ ]:
devices = nmct.box.list_onewire_ids()
print(devices)
if len(devices) == 1:
    t = nmct.box.get_thermometer(devices[0]).measure()
    print("It's {} \N{DEGREE SIGN}C".format(t))

21.0


In [3]:
import aiy.voicehat
from datetime import datetime
button = aiy.voicehat.get_button()
print("It's now {}".format(datetime.utcnow()))
button.wait_for_press()
print("You pressed at {}!".format(datetime.utcnow()))


You pressed!


## Watson Conversation

In [ ]:
import sys

from nmct import watson

credentials = {
    "url": "https://gateway-fra.watsonplatform.net/conversation/api",
    "username": "509bfc4e-9978-4fd7-8354-cad53285ecdf",
    "password": "TUpR7gIZUc1u"
}
workspace_id = "9de1dde4-e3a6-406b-9913-82e5ce74f64c"

cnv = watson.get_conversation(workspace_id, credentials)
intents, entities, output = cnv.message("Hey babe!")
print(output["text"][0])


In [ ]:


while not cnv.finished:
    print("\n------------------------------------------------------------------------------------")
    text = input()
    intents, entities, output = cnv.message(text)
    print("Intents: {} ".format(intents))
    print("Entities: {} ".format(entities))
    print("Output: {} ".format(output))
    print
    if "goodbyes" in intents:
        cnv.finish()


## Speech recognition

In [ ]:
from nmct import box, watson, Color, Palette
import aiy.voicehat

stt = watson.get_recognizer()
button = aiy.voicehat.get_button()
ring = box.get_pixel_ring()
running = True


def stop():
    running = False


button.on_press(stop)

while running:
    word = stt.recognize()
    
    if word in Palette:
        color = Color.by_name(word)
        ring.queue_effect("fill", color)
    else:
        nmct.box.play_sound("glass")
